<a href="https://colab.research.google.com/github/Jayashree-DataScience/cvfiles/blob/main/Image_Classification_Binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pathlib

import glob
import numpy as np
from sklearn.model_selection import train_test_split





In [3]:
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import *

In [4]:
def load_images_and_labels(image_paths):
    images = []
    labels = []

    for image_path in image_paths:
        image = load_img(image_path, target_size=(32, 32),
                         color_mode='grayscale')
        image = img_to_array(image)

        label = image_path.split(os.path.sep)[-2]
        label = 'positive' in label
        label = float(label)

        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)



In [5]:

def build_network():
    input_layer = Input(shape=(32, 32, 1))
    x = Conv2D(filters=20,
               kernel_size=(5, 5),
               padding='same',
               strides=(1, 1))(input_layer)
    x = ELU()(x) #What is Elu in neural network?
#Image result for elu deep learning
#The Exponential Linear Unit (ELU) is an activation function for neural networks. In contrast to ReLUs, ELUs have negative values which allows them to push mean unit activations closer to zero like batch normalization but with lower computational complexity.*/
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2),
                     strides=(2, 2))(x)
    x = Dropout(0.4)(x)

    x = Conv2D(filters=50,
               kernel_size=(5, 5),
               padding='same',
               strides=(1, 1))(x)
    x = ELU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2),
                     strides=(2, 2))(x)
    x = Dropout(0.4)(x)

    x = Flatten()(x)
    x = Dense(units=500)(x)
    x = ELU()(x)
    x = Dropout(0.4)(x)

    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=output)
    return model






In [6]:
#files_pattern = ('content'/'drive'/'MyDrive'/'keras-datasets'/'SMILEs' / '*' / '*' / '*.jpg')
files_pattern = ('/content/drive/MyDrive/keras-datasets/SMILEs/*/*/*.jpg')
files_pattern = str(files_pattern)
dataset_paths = [*glob.glob(files_pattern)]

X, y = load_images_and_labels(dataset_paths)

In [7]:
X /= 255.0
total = len(y)
total_positive = np.sum(y)
total_negative = total - total_positive
print(f'Total images: {total}')
print(f'Smile images: {total_positive}')
print(f'Non-smile images: {total_negative}')



Total images: 13158
Smile images: 3690.0
Non-smile images: 9468.0


In [8]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y,
                                     test_size=0.2,
                                     stratify=y,
                                     random_state=999)
(X_train, X_val,
 y_train, y_val) = train_test_split(X_train, y_train,
                                    test_size=0.2,
                                    stratify=y_train,
                                     random_state=999)
                                   

model = build_network()
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

BATCH_SIZE = 32
EPOCHS = 20
model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          class_weight={
              1.0: total / total_positive,
              0.0: total / total_negative
          })
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {test_loss}, accuracy: {test_accuracy}')

Epoch 1/20
264/264 [==============================] - 31s 114ms/step - loss: 1.9087 - accuracy: 0.7379 - val_loss: 2.6272 - val_accuracy: 0.5024
Epoch 2/20
264/264 [==============================] - 30s 113ms/step - loss: 0.9776 - accuracy: 0.8227 - val_loss: 0.4581 - val_accuracy: 0.8637
Epoch 3/20
264/264 [==============================] - 34s 130ms/step - loss: 0.8473 - accuracy: 0.8425 - val_loss: 0.3105 - val_accuracy: 0.8737
Epoch 4/20
264/264 [==============================] - 30s 114ms/step - loss: 0.7603 - accuracy: 0.8533 - val_loss: 0.2837 - val_accuracy: 0.8884
Epoch 5/20
264/264 [==============================] - 30s 114ms/step - loss: 0.6788 - accuracy: 0.8694 - val_loss: 0.2856 - val_accuracy: 0.8818
Epoch 6/20
264/264 [==============================] - 30s 113ms/step - loss: 0.6144 - accuracy: 0.8806 - val_loss: 0.3059 - val_accuracy: 0.8727
Epoch 7/20
264/264 [==============================] - 30s 113ms/step - loss: 0.5951 - accuracy: 0.8884 - val_loss: 0.2159 - val_ac